**Sommario**
- [Iterabili, iteratori, generatori](#iterabili-iteratori-generatori-enumerate-range)
  - [In sintesi](#in-sintesi)
  - [Lazy evaluation](#lazy-evaluation)
  - [`<class 'generator'>`](#<class-'generator'>)
    - [Generator expression](#generator-expression)
    - [Generator function](#generator-function)
  - [`<class '*_iterator'>`](#<class-'*_iterator'>)
  - [`<class 'enumerate'>`](#<class-'enumerate'>)
  - [*Iterator protocol*: implementare un oggetto iteratore](#*iterator-protocol*-implementare-un-oggetto-iteratore)
  - [Domanda: Ma li iteratori possono essere "resettati"?](#domanda-ma-li-iteratori-possono-essere-resettati?)
  - [Comparazione tra iteratori e generatori](#comparazione-tra-iteratori-e-generatori)
  - [Iteratore con "retromarcia"](#iteratore-con-retromarcia)

# Iterabili, iteratori, generatori

![iterables.png](./imgs/iterables.png)

## In sintesi

- un _**iterabile**_ è un termine generico che indica qualcosa su cui si può iterare;

- un _**iteratore**_ è un termine generico che indica un oggetto che possiede dei metodi che consentono l'iterazione, definiti dal cosiddetto *iterator protocol*;

- un _**generatore**_ è un modo comodo per creare iteratori senza dover implementare una classe ed esplicitare i metodi dell'*iterator protocol*.

| Concetto         | Spiegazione                                                                                                                                                                                                            |
|------------------|------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| iterable         | Un oggetto in grado di restituire i suoi membri uno alla volta, e sul quale è possibile eseguire un ciclo e che può essere trasformato in un iteratore. Esempi di iterabili sono `list`, `set`, `tuple`, `dict`, `string`, ecc.                                                |
| iterator         | Un oggetto che rappresenta uno *stream* (flusso) di dati e che può essere iterato e che implementa i metodi `__iter__` e `__next__`.                                                                                                                                                                                     |
| generator        | Un tipo speciale di funzione che non restituisce un singolo valore, ma restituisce un oggetto iteratore che "possiede" una sequenza di valori.                                                                                       |
| lazy evaluation  | Una strategia di valutazione per cui alcuni oggetti vengono prodotti solo quando necessario. Di conseguenza, alcuni circoli di sviluppatori si riferiscono alla valutazione pigra anche come "call-by-need".           |
| iterator protocol | Un insieme di regole da seguire per implementare un iteratore in Python.                                                                                                                                               |
| `next()`         | Una funzione built-in utilizzata per restituire l'elemento successivo in un iteratore. L'oggetto deve implementare il metodo `__next__`.                                                                               |
| `iter()`         | Una funzione built-in usata per convertire un iterabile in un iteratore. L'oggetto deve implementare il metodo `__iter__`.                                                                                             |
| `for`            | Il ciclo `for` utilizza il metodo `__iter__` per ottenere un iteratore correttamente inizializzato e `__next__` per esegiuire l'iterazione. Quando rileva un errore di `StopIteration`, termina automaticamente il ciclo senza effetti collaterali. |
| `yield`          | Una keyword di Python simile alla parola chiave `return`, con la differenza che `yield` restituisce un oggetto generatore invece di un valore.                                                                         |

![iterable_iterator_generator.png](./imgs/iterable_iterator_generator.png)

## Lazy evaluation

Vedi [su Wikipedia](https://it.wikipedia.org/wiki/Valutazione_lazy).

## `<class 'generator'>` 

### Generator expression

La forma più compatta per creare un generatore è detta [_**Generator expressions**_](https://docs.python.org/3/reference/expressions.html#generator-expressions) (*genexp*).

Una *genex* ha la forma `(<exp> for <element> in <iterable>)`.

In [ ]:
# gen = (x for x in range(3))
gen = (x for x in ['A', 'B', 'C'])

print(type(gen))

# print(gen.__next__())
# print(gen.__next__())
# print(gen.__next__())
# print(gen.__next__())  # StopIteration error

print(next(gen))
print(next(gen))
print(next(gen))
# print(next(gen))  # StopIteration error

# for x in gen:
#     print(x)

<class 'generator'>
A
B
C


A prima vista potrebbe sembrare una "tuple comprehension", ma in questo caso le parentesi tonde non ci devono trarre in inganno. Questo è un **generatore** a tutti gli effetti.

> CURIOSITÀ: La *tuple comprehension* la si ottiene così:<br/>
  `tuple(x for x in ['A', 'B', 'C'])`<br/>
  In pratica gli si passa un generatore come argomento.

### Generator function

Un altro modo per implementare un generatore è utilizzando la parola chiave `yeld` all'interno di una funzione (al posto di `return`).

In [15]:
def generator():
    yield 1
    yield 2
    yield 3

gen = generator()

print(type(gen))

print(next(gen))
print(next(gen))
print(next(gen))
# print(next(gen))  # StopIteration error

<class 'generator'>
1
2
3


Questo è un generatore perché `generator()` è una funzione che produce una sequenza di valori. Utilizzando `yield`, ogni chiamata a `next()` sul generatore restituirà il prossimo valore nella sequenza.

Possiamo generare una sequenza di numeri in modo dinamico e iterarci sopra:

In [12]:
def my_generator_range(len):
    item = 0
    yield 1
    item += 1
    while item < len:
        yield 2
        item += 1

gen = my_generator_range(3)

print(type(gen))

print(next(gen))
print(next(gen))
print(next(gen))
# print(next(gen))  # StopIteration error

<class 'generator'>
1
2
2


Oppure possiamo creare un generatore che itera sopra un iterabile e modifica i suoi elementi.

Il vantaggio dei generatori ci consente di eseguire i calcoli solo solo quando è necessario e senza duplicare l'elemento originale. Questo fa risparmiare potenza di calcolo e memoria.

Per esempio potremmo trasformare in minuscole le nostre lettere dell'esempio precedente:

In [ ]:
def my_generator_iterable(iterable):
    lenght = len(iterable)
    idx = 0
    item = iterable[0].lower()
    yield item
    idx += 1
    while idx < lenght:
        yield iterable[idx].lower()
        idx += 1


gen = my_generator_iterable(['A', 'B', 'C'])

print(type(gen))

print(next(gen))
print(next(gen))
print(next(gen))
# print(next(gen))  # StopIteration error

<class 'generator'>
a
b
c


Naturalmente, invece di "iterare" semplicemente passo-passo un iterabile o un range, potremmo eseguire eseguire dei calcoli aggiuntivi ad ogni iterazione e restituire il risultato di quel calcolo anziché l'elemento originale.

In [1]:
def squares(start, stop):
    for i in range(start, stop):
        yield i * i

gen = squares(1, 5)
print(type(gen))

print(next(gen))
print(next(gen))
print(next(gen))
print(next(gen))
# print(next(gen))  # StopIteration error

<class 'generator'>
1
4
9
16


In [2]:
gen = (i*i for i in range(1, 5))
print(type(gen))

print(next(gen))
print(next(gen))
print(next(gen))
print(next(gen))
# print(next(gen))  # StopIteration error

<class 'generator'>
1
4
9
16


## `<class '*_iterator'>`

Per creare un iteratore a partire da un iterabile, possiamo semplicemente usare la funzione built-in `iter()`.

In [4]:
# iterabile = range(3)
iterabile = ['A', 'B', 'C']

iteratore = iter(iterabile)

print(type(iteratore))

print(next(iteratore))
print(next(iteratore))
print(next(iteratore))
# print(next(iteratore))  # StopIteration error

<class 'list_iterator'>
A
B
C


## `<class 'enumerate'>`

La funzione `enumerate()` è simile a `iter()` ma ci restituisce una tupla (indice, valore).

In [4]:
enum = enumerate(['A', 'B', 'C'])
print(type(enum))

print(next(enum))
print(next(enum))
print(next(enum))
# print(next(enum))  # StopIteration error

# for idx, elem in enum:
#     print('index:', idx, 'element:', elem)

<class 'enumerate'>
(0, 'A')
(1, 'B')
(2, 'C')


## *Iterator protocol*: implementare un oggetto iteratore

Un oggetto iteratore deve implementare il metodo speciale `__iter__` che deve restituire un iterabile e quello `__next__` che deve restituire un elemento alla volta fino all'esaurimento dell'iterabile.

> APPROFONDIMENTO: Il metodo `__iter__` viene richiamato quando è necessario ottenere un iteratore a partire da un container. Questo metodo deve restituire un nuovo oggetto iteratore che può iterare su tutti gli oggetti del contenitore. Per le mappature, deve iterare sulle chiavi del contenitore.

Per fare questo e creare un oggetto che si comporta a tutti gli effetti come un iteratore, possiamo fare così:

In [6]:
class MyFullIterator():
    def __init__(self, stop):
        self._stop = stop
        self._cursore = 0  # inizializzo il contatore del "cursore"

    def __iter__(self):
        return self        # restituisce l'oggetto creato

    def __next__(self):
        if self._cursore < self._stop:
            res = self._cursore
            self._cursore += 1
            return res
        else:
            raise StopIteration

my_iterator = MyFullIterator(3)
print(type(my_iterator))

my_init_iterator = iter(my_iterator)
print(type(my_init_iterator))

print(next(my_init_iterator))
print(next(my_init_iterator))
print(next(my_init_iterator))
# print(next(my_init_iterator))  # StopIteration error

# for x in my_iterator:
#     print(x)

<class '__main__.MyFullIterator'>
<class '__main__.MyFullIterator'>
0
1
2


## Domanda: Ma li iteratori possono essere "resettati"?

Gli iteratori di Python normalmente non possono essere "resettati": una volta esauriti, dovrebbero sollevare una `StopIteration` ogni volta che viene chiamato `next()`. Per iterare nuovamente è necessario richiedere un nuovo oggetto iteratore con la funzione `iter()`.

## Comparazione tra iteratori e generatori

1. **Implementazione**:
   - Gli **iteratori** sono implementati tramite classi che definiscono i metodi `__iter__()` e `__next__()`.
   - I **generatori** sono implementati tramite funzioni che utilizzano la parola chiave `yield`.

2. **Stato e variabili locali**:
   - Negli **iteratori**, lo stato deve essere esplicitamente mantenuto e gestito all'interno della classe.
   - Nei **generatori**, lo stato e le variabili locali vengono automaticamente mantenuti tra le chiamate di `yield`.

3. **Utilizzo**:
   - Gli **iteratori** sono utili per creare oggetti che permettono l'iterazione su collezioni di dati o sequenze personalizzate.
   - I **generatori** sono particolarmente efficienti per calcolare sequenze di dati "on the fly" senza la necessità di memorizzare l'intera sequenza in memoria.

4. **Funzioni e parole chiave**:
   - Gli **iteratori** utilizzano i metodi `__iter__()` per restituire l'iteratore stesso e `__next__()` per accedere al prossimo elemento.
   - I **generatori** utilizzano la parola chiave `yield` per restituire il prossimo valore nella sequenza.

5. **Relazione**:
   - Tutti i **generatori** sono **iteratori**, poiché implementano i metodi `__iter__()` e `__next__()` implicitamente attraverso l'uso di `yield`.
   - Non tutti gli **iteratori** sono **generatori**, dato che gli iteratori possono essere creati senza utilizzare `yield`, tramite l'implementazione manuale dei metodi richiesti.


| Aspetto                | Iteratori                                      | Generatori                            |
|------------------------|------------------------------------------------|---------------------------------------|
| Implementazione        | Tramite classi con `__iter__()` e `__next__()` | Tramite funzioni con `yield`          |
| Gestione dello *stato* | Esplicita, all'interno della classe            | Implicita, tra le chiamate di `yield` |
| Utilizzo               | Iterazione su collezioni/sequenze              | Calcolo "on the fly" di sequenze      |
| Funzioni/Parole chiave | `__iter__()`, `__next__()`                     | `yield`                               |
| Relazione              | Possono essere generatori se usano `yield`     | Sono sempre iteratori                 |


## Iteratore con "retromarcia"

Qualcuno potrebbe chiedersi perché dover ricorrere a un iteratore se con l'uso della keyword `yield` possiamo creare generatori in modo più facile e veloce.

I generatori vanno bene per la maggioranza dei casi ma, per esempio, se volessimo implementare un oggetto che possa iterare in modo bidirezionale, ovvero sia in avanti che indietro, saremmo costretti a implementare un iteratore ad hoc.

Nell'esempio che segue si implementa un iteratore con un metodo per attivare una sorta di "retromarcia" che consente di invertire la direzione dell'iterazione.

In [7]:
class BidirectionalIteratorWithReverse:
    def __init__(self, data):
        self.data = data
        self.index = 0
        self.forward = True  # Direzione di default: avanti

    def __iter__(self):
        return self

    def __next__(self):
        # Se si sta andando avanti
        if self.forward:
            # Se non si è arrivati alla fine della lista
            if self.index < len(self.data):
                # Si restituisce l'elemento corrente e si incrementa l'indice
                result = self.data[self.index]
                self.index += 1
                return result
            # Se si è arrivati alla fine della lista, ferma l'iterazione
            else:
                raise StopIteration
        # Se si sta andando all'indietro
        else:
            # Se non si è arrivati all'inizio della lista
            if self.index > 0:
                # Si decrementa l'indice e si restituisce l'elemento corrente
                self.index -= 1
                return self.data[self.index]
            # Se si è arrivati all'inizio della lista, ferma l'iterazione
            else:
                raise StopIteration

    def reverse_direction(self):
        # Inverte la direzione dell'iterazione
        self.forward = not self.forward

        # Si aggiusta l'indice in base alla nuova direzione in modo da non
        # saltare elementi
        if self.forward:
            self.index -= 1
        else:
            self.index += 1


# Esempio di utilizzo
data = [1, 2, 3, 4, 5]
iteratore = BidirectionalIteratorWithReverse(data)


print(next(iteratore))
print(next(iteratore))
iteratore.reverse_direction()  # Attiva la retromarcia
print(next(iteratore))
print(next(iteratore))
print(next(iteratore))

1
2
3
2
1
